In [3]:
import pandas as pd
import numpy as np

# 1. LOAD RAW DATA

flights = pd.read_csv("flights.csv")
hotels = pd.read_csv("hotels.csv")
users = pd.read_csv("users.csv")

# Standardise key column names
flights = flights.rename(columns={"userCode": "user_id", "travelCode": "trip_id"})
hotels  = hotels.rename(columns={"userCode": "user_id", "travelCode": "trip_id"})
users   = users.rename(columns={"code": "user_id"})


# 2. BASIC CLEANING

# Convert date columns
if "date" in flights.columns:
    flights["date"] = pd.to_datetime(flights["date"])
if "date" in hotels.columns:
    hotels["date"] = pd.to_datetime(hotels["date"])

# Numeric conversions
for col in ["price", "time", "distance"]:
    if col in flights.columns:
        flights[col] = pd.to_numeric(flights[col], errors="coerce")

for col in ["price", "total", "days"]:
    if col in hotels.columns:
        hotels[col] = pd.to_numeric(hotels[col], errors="coerce")

# Merge user info in (optional but nice to have)
flights_full = flights.merge(users, on="user_id", how="left")
hotels_full  = hotels.merge(users,  on="user_id", how="left")


# 3. AGGREGATE TO TRIP LEVEL

# Flights: group by trip, user, AND agency so we keep supplier info
flight_cost_per_trip = (
    flights_full
    .groupby(["trip_id", "user_id", "agency"], dropna=False)
    .agg(
        flight_total    = ("price", "sum"),
        flight_distance = ("distance", "sum"),
        flight_time     = ("time", "sum"),
        flights_count   = ("price", "count")
    )
    .reset_index()
)

# Hotels: group by trip + user (no agency here)
hotel_cost_per_trip = (
    hotels_full
    .groupby(["trip_id", "user_id"], dropna=False)
    .agg(
        hotel_total    = ("total", "sum"),
        hotel_nights   = ("days", "sum"),
        hotel_bookings = ("total", "count")
    )
    .reset_index()
)

# Merge flights + hotels
trips = flight_cost_per_trip.merge(
    hotel_cost_per_trip,
    on=["trip_id", "user_id"],
    how="left"
)

# Fill NaNs for hotel fields where there was no hotel stay
trips[["hotel_total", "hotel_nights", "hotel_bookings"]] = trips[
    ["hotel_total", "hotel_nights", "hotel_bookings"]
].fillna(0)


# 4. CORE METRICS

trips["total_trip_cost"] = trips["flight_total"] + trips["hotel_total"]

# Cost per km
trips["cost_per_km"] = trips["flight_total"] / trips["flight_distance"]
trips["cost_per_km"] = trips["cost_per_km"].replace([np.inf, -np.inf], np.nan).round(2)

# Cost per hotel night
trips["cost_per_night"] = trips["hotel_total"] / trips["hotel_nights"]
trips["cost_per_night"] = trips["cost_per_night"].replace([np.inf, -np.inf], np.nan).round(2)

# Cost per flight segment
trips["cost_per_flight"] = trips["flight_total"] / trips["flights_count"]
trips["cost_per_flight"] = trips["cost_per_flight"].round(2)

# Trip duration approximation (use hotel nights where available)
trips["trip_duration"] = trips["hotel_nights"]
mask_zero = trips["trip_duration"] == 0
trips.loc[mask_zero, "trip_duration"] = (trips.loc[mask_zero, "flight_time"] / 2).round(1)


# 5. ANOMALY FLAG + CO2

# Z-score on total trip cost
trips["cost_zscore"] = (
    (trips["total_trip_cost"] - trips["total_trip_cost"].mean())
    / trips["total_trip_cost"].std()
)

# Flag very expensive trips
trips["flag_high_cost"] = trips["cost_zscore"] > 2.5

# Simple CO2 estimation
CO2_PER_KM_FLIGHT  = 0.15  # kg CO2 per km (approx)
CO2_PER_NIGHT_HOTEL = 30   # kg CO2 per night (approx)

trips["co2_flight"] = trips["flight_distance"] * CO2_PER_KM_FLIGHT
trips["co2_hotel"]  = trips["hotel_nights"] * CO2_PER_NIGHT_HOTEL
trips["co2_total"]  = (trips["co2_flight"] + trips["co2_hotel"]).round(2)


# 6. EXPORT CLEAN DATASET
trips.to_csv("clean_trips_enhanced2.csv", index=False)

print("Saved file: clean_trips_enhanced.csv")
display(trips.head())
print("\nColumns:", trips.columns.tolist())


Saved file: clean_trips_enhanced.csv


,trip_id,user_id,agency,flight_total,flight_distance,flight_time,flights_count,hotel_total,hotel_nights,hotel_bookings,total_trip_cost,cost_per_km,cost_per_night,cost_per_flight,trip_duration,cost_zscore,flag_high_cost,co2_flight,co2_hotel,co2_total
0,0,0,FlyingDrops,2726.67,1353.06,3.52,2,1252.08,4.0,1.0,3978.75,2.02,313.02,1363.34,4.0,2.579506,True,202.959,120.0,322.96
1,1,0,CloudFy,2614.88,1275.12,3.32,2,0.00,0.0,0.0,2614.88,2.05,NaN,1307.44,1.7,0.731800,False,191.268,0.0,191.27
2,2,0,CloudFy,3215.97,1661.72,4.32,2,526.82,2.0,1.0,3742.79,1.94,263.41,1607.98,2.0,2.259839,False,249.258,60.0,309.26
3,3,0,Rainbow,1621.10,1300.20,3.38,2,0.00,0.0,0.0,1621.10,1.25,NaN,810.55,1.7,-0.614525,False,195.030,0.0,195.03
4,4,0,Rainbow,1498.69,1353.06,3.52,2,0.00,0.0,0.0,1498.69,1.11,NaN,749.34,1.8,-0.780360,False,202.959,0.0,202.96



Columns: ['trip_id', 'user_id', 'agency', 'flight_total', 'flight_distance', 'flight_time', 'flights_count', 'hotel_total', 'hotel_nights', 'hotel_bookings', 'total_trip_cost', 'cost_per_km', 'cost_per_night', 'cost_per_flight', 'trip_duration', 'cost_zscore', 'flag_high_cost', 'co2_flight', 'co2_hotel', 'co2_total']
